# Challange Bestseller

##### Librerias necesarias

In [344]:
import requests as rq
import json
import numpy as np
import pandas as pd
import datetime
from datetime import datetime as dt

### API search Mercado Libre Brasil

In [345]:
APIsearch = 'https://api.mercadolibre.com/sites/MLB/search'
max_results = 200
pagLimit = 50
query = 'samsung galaxy a20'
condition = 'new'
category = 'MLB1055' # id para celulares e Smarphones
responseTimeSearch = []
id_item = []

x = 0
while x < max_results:
    # While que maneja la paginacion
    args = {'q': query, 'condition': condition, 'category': category, 'offset': x, 'limit': pagLimit}
    response = rq.get(APIsearch, params = args)
    responseTimeSearch.append(response.elapsed.total_seconds())
    data = json.loads(response.content)
    for i in data['results']:
        # ciclo para guardar en un array los id_items recopilados de cada invocacion a la API
        id_item.append( i['id'])
    x += pagLimit

### API para obtener los atributos de cada items

In [346]:
def warranty_translate (sale_terms):
    if len(sale_terms) != 0:
        return sale_terms[0]['id']
    else:
        return 'Not Warranty'

In [347]:
APIitems = 'https://api.mercadolibre.com/items/'
responseTimeItems = []
sold_quantity=[]
start_time=[]
id_seller=[] 
prices=[]
warranty=[]
mode_shipping=[]
international_delivery_mode = []
free_shipping = []

for i in id_item:
    responseItems = rq.get(APIitems + i)
    responseTimeItems.append(responseItems.elapsed.total_seconds()) #tiempo de respuesta cuando voy a pegarle a la API
    item = json.loads(responseItems.content)
    #Llenando listas con datos para luego insertar en el DataFrame
    sold_quantity.append(item['sold_quantity'])
    start_time.append(item['start_time'])
    id_seller.append(item['seller_id'])
    prices.append(item['price'])
    warranty.append(warranty_translate(item['sale_terms']))
    mode_shipping.append(item['shipping']['mode'])
    free_shipping.append(item['shipping']['free_shipping'])   
    international_delivery_mode.append(item['international_delivery_mode'])    

#### API conversion currency Brasil 'BRL' to dollar 'USD'

In [349]:
responseCurrencyConversions = rq.get('https://api.mercadolibre.com/currency_conversions/search?from=BRL&to=USD')
responseCurrencyConversions.elapsed.total_seconds()
conversion = json.loads(responseCurrencyConversions.content)

In [351]:
df = pd.DataFrame({'id_item':id_item, 'id_seller':id_seller,'sold_quantity':sold_quantity,
                   'diff_days':[str(datetime.date.today()-(dt.strptime(x[:-14],'%Y-%m-%d').date()))[:-14] for x in start_time], 
                   'price_dollar':[ np.around(x*conversion['ratio'],2) for x in prices], 'warranty':warranty,
                   'mode_shipping':mode_shipping, 'free_shipping':free_shipping, 
                   'international_delivery':international_delivery_mode})
df.head()

,id_item,id_seller,sold_quantity,diff_days,price_dollar,warranty,mode_shipping,free_shipping,international_delivery
0,MLB1388367860,179890703,0,98,200.78,WARRANTY_TIME,me1,False,none
1,MLB1388367879,321921696,0,98,233.07,Not Warranty,me2,True,none
2,MLB1462418937,155425567,25,11,186.42,WARRANTY_TYPE,me2,True,none
3,MLB1465272277,135482855,1,9,200.22,WARRANTY_TYPE,me2,True,none
4,MLB1219240233,155425567,500,329,252.51,WARRANTY_TYPE,me1,False,none
